### Class Targets are given

In [1]:
import numpy as np

In [2]:
softmaxOutputs = np.array([
    [0.7, 0.1, 0.2],
    [0.1, 0.5, 0.4],
    [0.02, 0.9, 0.08]
])

classTargets = np.array(
    [0, 1, 1]
)

In [3]:
print(softmaxOutputs[range(len(softmaxOutputs)), classTargets])

[0.7 0.5 0.9]


In [4]:
loss = -np.log(softmaxOutputs[range(len(softmaxOutputs)), classTargets])
print(f'Loss: {loss}')
avg_loss = np.mean(loss)
print(f'Average Loss: {avg_loss}')

Loss: [0.35667494 0.69314718 0.10536052]
Average Loss: 0.38506088005216804


### One Hot Encoding

In [5]:
softmaxOutputs = np.array([
    [0.7, 0.1, 0.2],
    [0.1, 0.5, 0.4],
    [0.02, 0.9, 0.08]
])

classTargets = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 1, 0]
])

In [6]:
clipped = softmaxOutputs * classTargets
clipped = np.sum(clipped, axis=1)
print(clipped)

[0.7 0.5 0.9]


In [7]:
loss = -np.log(clipped)
avg_loss = np.mean(loss)
print(f'Loss: {loss}')
print(f'Average Loss: {avg_loss}')

Loss: [0.35667494 0.69314718 0.10536052]
Average Loss: 0.38506088005216804


### The Categorical Cross Entropy (CCE) Class

In [8]:
class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        delta_loss = np.mean(sample_losses)
        return delta_loss

In [9]:
class CCE(Loss):
    def forward(self, y_pred, y_true): # y_pred -> prediction matrix, y_true -> classTargets
        samples = len(y_pred)
        y_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        if len(y_true.shape) == 1: # only if categorical labels
            # y_true.ndim == 1 also works
            confidences = y_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2: # one hot encoding
            # y_true.ndim == 2 also works
            confidences = np.sum(y_clipped * y_true, axis=1)
        negativeLossLikelihood = -np.log(confidences)
        return negativeLossLikelihood

### Example Loss

In [10]:
softmaxOutputs = np.array([
    [0.7, 0.1, 0.2],
    [0.1, 0.5, 0.4],
    [0.02, 0.9, 0.08]
])

classTargets = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 1, 0]
])

lossFunction = CCE()
loss = lossFunction.calculate(softmaxOutputs, classTargets)
print(loss)

0.38506088005216804


### A Complete Pass through the network with the Loss

In [11]:
import numpy as np
from nnfs.datasets import spiral_data
import nnfs

In [12]:
nnfs.init()

class DenseLayer:
    def __init__(self, n_inputs, n_neurons):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.outputs = np.dot(inputs, self.weights) + self.biases

class ReLU:
    def forward(self, inputs):
        self.outputs = np.maximum(0, inputs)

class Softmax:
    def forward(self, inputs):
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        self.outputs = exp_values / np.sum(exp_values, axis=1, keepdims=True)

class Loss:
    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        delta_loss = np.mean(sample_losses)
        return delta_loss

class CCE(Loss):
    def forward(self, y_pred, y_true): # y_pred -> prediction matrix, y_true -> classTargets
        samples = len(y_pred)
        y_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        if len(y_true.shape) == 1: # only if categorical labels
            # y_true.ndim == 1 also works
            confidences = y_clipped[range(samples), y_true]
        elif len(y_true.shape) == 2: # one hot encoding
            # y_true.ndim == 2 also works
            confidences = np.sum(y_clipped * y_true, axis=1)
        negativeLossLikelihood = -np.log(confidences)
        return negativeLossLikelihood

x, y = spiral_data(samples=100, classes=3)

In [13]:
relu = ReLU() # Create objects of activation functions and loss functions
softmax = Softmax()
lossFunction = CCE()

In [14]:
layer1 = DenseLayer(2, 3)
layer1.forward(x)
relu.forward(layer1.outputs)
layer2 = DenseLayer(3, 3)
layer2.forward(relu.outputs)
softmax.forward(layer2.outputs)

In [15]:
print(softmax.outputs[:5])

[[0.33333334 0.33333334 0.33333334]
 [0.3333332  0.3333332  0.33333364]
 [0.3333329  0.33333293 0.3333342 ]
 [0.3333326  0.33333263 0.33333477]
 [0.33333233 0.3333324  0.33333528]]


In [16]:
loss = lossFunction.calculate(softmax.outputs, y) # softmax.outputs -> prediction and y -> classTargets

predictions = np.argmax(softmax.outputs, axis=1)
if len(y.shape) == 2:
    y = np.argmax(y, axis=1)
accuracy = np.mean(predictions == y)

print(accuracy)
print(loss)

0.34
1.0986104
